# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("02-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'14 17 20 21 24 26 27 24\n39 41 43 45 46 46\n35 38 39 41 44 47 50 54\n68 69 71 74 75 78 80 87\n80 82 81 82 83 85 88\n48 51 54 55 58 57 55\n41 44 47 50 47 47\n66 68 71 70 73 77\n29 32 29 30 35\n77 78 79 81 83 83 86 88\n10 13 14 16 19 19 20 17\n61 64 65 67 67 67\n29 30 31 32 35 35 39\n24 25 25 28 31 38\n61 64 65 66 70 73 76\n35 37 39 43 40\n41 42 43 47 48 49 49\n3 5 9 11 15\n24 26 28 32 34 35 40\n48 49 50 57 59 62 64 67\n18 20 22 25 26 32 35 33\n76 79 85 86 86\n55 58 59 66 70\n9 11 13 14 20 23 29\n58 56 58 59 60 62 63 65\n42 39 40 41 42 43 46 43\n83 80 81 83 86 89 89\n48 47 48 50 52 56\n38 37 38 39 46\n54 53 56 53 55 56\n79 76 74 77 76\n68 65 63 64 67 70 70\n84 81 83 86 84 87 91\n72 71 74 72 78\n38 35 36 36 38 41 43 46\n45 43 45 47 49 49 50 48\n23 22 24 24 24\n9 6 8 10 12 12 13 17\n77 75 78 78 79 80 83 89\n60 58 62 65 66 69 71\n64 62 63 64 68 67\n76 73 76 77 81 83 83\n21 19 21 25 26 29 31 35\n53 52 53 54 58 61 63 70\n91 89 94 97 99\n15 14 17 23 24 25 22\n65 62 65 71 74 75 75\n79 76 77 83

In [10]:
test_data_raw = r"""7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9"""


def preprocess_data (data):
    # dtype='U10'
    reports = []
    for row in data.split("\n"):   
        reports.append(np.array([int(item) for item in row.split(" ")]))

    return reports
test_data = preprocess_data(test_data_raw)
display(test_data)

[array([7, 6, 4, 2, 1]),
 array([1, 2, 7, 8, 9]),
 array([9, 7, 6, 2, 1]),
 array([1, 3, 2, 4, 5]),
 array([8, 6, 4, 4, 1]),
 array([1, 3, 6, 7, 9])]

In [11]:
data = preprocess_data(data_raw)
data

[array([14, 17, 20, 21, 24, 26, 27, 24]),
 array([39, 41, 43, 45, 46, 46]),
 array([35, 38, 39, 41, 44, 47, 50, 54]),
 array([68, 69, 71, 74, 75, 78, 80, 87]),
 array([80, 82, 81, 82, 83, 85, 88]),
 array([48, 51, 54, 55, 58, 57, 55]),
 array([41, 44, 47, 50, 47, 47]),
 array([66, 68, 71, 70, 73, 77]),
 array([29, 32, 29, 30, 35]),
 array([77, 78, 79, 81, 83, 83, 86, 88]),
 array([10, 13, 14, 16, 19, 19, 20, 17]),
 array([61, 64, 65, 67, 67, 67]),
 array([29, 30, 31, 32, 35, 35, 39]),
 array([24, 25, 25, 28, 31, 38]),
 array([61, 64, 65, 66, 70, 73, 76]),
 array([35, 37, 39, 43, 40]),
 array([41, 42, 43, 47, 48, 49, 49]),
 array([ 3,  5,  9, 11, 15]),
 array([24, 26, 28, 32, 34, 35, 40]),
 array([48, 49, 50, 57, 59, 62, 64, 67]),
 array([18, 20, 22, 25, 26, 32, 35, 33]),
 array([76, 79, 85, 86, 86]),
 array([55, 58, 59, 66, 70]),
 array([ 9, 11, 13, 14, 20, 23, 29]),
 array([58, 56, 58, 59, 60, 62, 63, 65]),
 array([42, 39, 40, 41, 42, 43, 46, 43]),
 array([83, 80, 81, 83, 86, 89, 89])

array([0, 0, 0])

In [26]:


def solution (data, verbose=False):

    safe_reports = 0
    for report in data:
        forward = report[1:] - report[:-1]
        if (np.abs(forward) <=3).all() and ((forward > 0).all() or (forward < 0).all()):
            safe_reports += 1

    return safe_reports



sol = solution(data)
# sol = solution(test_data, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))


202


# Part 2

In [37]:
def solution2(data, verbose=False):

    def check_safeness(rep):
        forward = rep[1:] - rep[:-1]
        if (np.abs(forward) <=3).all() and ((forward > 0).all() or (forward < 0).all()):
            return True
        else:
            return False

    safe_reports = 0
    for report in data:
        
        if check_safeness(report):
            safe_reports += 1
        else:
            for i in range(report.shape[0]):
                if check_safeness(np.delete(report, i)):
                    safe_reports += 1
                    break
        

    return safe_reports



sol = solution2(data)
# sol = solution2(test_data, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))

    

271
